In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import math
import yaml

%matplotlib inline

In [2]:
dataFilePathNN = "../data/testLibrary/results1/Cijk_Ailk_Bljk_SB/0/data/benchmark.csv"
dataNN = pd.read_csv(dataFilePathNN).rename(str.strip,axis='columns')

dataFilePathNT = "../data/testLibrary/results1/Cijk_Ailk_Bjlk_SB/1/data/benchmark.csv"
dataNT = pd.read_csv(dataFilePathNT).rename(str.strip,axis='columns')

dataFilePathTN = "../data/testLibrary/results1/Cijk_Alik_Bljk_SB/2/data/benchmark.csv"
dataTN = pd.read_csv(dataFilePathTN).rename(str.strip,axis='columns')

In [3]:
dataNN.head()

,GFlops,SizeI,SizeJ,SizeK,SizeL,LDD,LDC,LDA,LDB,TotalFlops,...,Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS1_GRVW4_K1_LPB4_NLCA1_PGR1_WG32_4_1_WGM1,Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS1_GRVW4_K1_LPB0_NLCA1_PGR1_WG32_4_2_WGM8,Cijk_Ailk_Bljk_SB_MT256x32x8_SE_EPS1_GRVW1_K1_LPB4_NLCA2_PGR1_WG32_4_1_WGM8,Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW4_K1_LPB0_NLCA1_PGR0_WG32_4_2_WGM8,Cijk_Ailk_Bljk_SB_MT256x32x8_SE_EPS0_GRVW4_K1_LPB0_NLCA1_PGR0_WG32_4_2_WGM8,Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW4_K1_LPB4_NLCA1_PGR0_WG32_4_1_WGM8,Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW1_K1_LPB0_NLCA2_PGR0_WG32_4_1_WGM8,Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW4_K1_LPB0_NLCA1_PGR0_WG32_4_1_WGM8,Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW1_K1_LPB0_NLCA1_PGR0_WG32_4_2_WGM1,Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW4_K1_LPB0_NLCA1_PGR0_WG32_4_2_WGM8
0,0,192,240,1,256,192,192,192,256,23592960,...,184,271,182,278,254,195,173,193,260,273
1,1,192,256,1,256,192,192,192,256,25165824,...,196,193,195,298,273,209,193,210,274,290
2,2,192,300,1,256,192,192,192,256,29491200,...,233,344,228,5,318,246,225,240,324,343
3,3,193,240,1,256,193,193,193,256,23715840,...,189,253,185,280,249,194,186,194,260,268
4,4,193,256,1,256,193,193,193,256,25296896,...,200,295,197,297,272,209,197,208,274,290


In [4]:
solutionFileNN="../data/testLibrary/Cijk_Ailk_Bljk_SB/0/solutions/solutions.yaml"
streamNN = open(solutionFileNN, "r")
solutionsD_NN = yaml.load(streamNN, yaml.SafeLoader)
streamNN.close()

solutionFileNT="../data/testLibrary/Cijk_Ailk_Bjlk_SB/1/solutions/solutions.yaml"
streamNT = open(solutionFileNT, "r")
solutionsD_NT = yaml.load(streamNT, yaml.SafeLoader)
streamNT.close()

solutionFileTN="../data/testLibrary/Cijk_Alik_Bljk_SB/2/solutions/solutions.yaml"
streamTN = open(solutionFileTN, "r")
solutionsD_TN = yaml.load(streamTN, yaml.SafeLoader)
streamTN.close()

In [5]:
def nTiles (MT, Size):
    nTiles = float(Size)/float(MT)
    return math.ceil(nTiles)

def TileGranularity (MT, Tiles, Size):
    tileGranularity = (float(Size) / float(MT)) / float(Tiles)
    return tileGranularity

def TilesPerCu (nBatches, Tiles0, Tiles1, nCUs, GSU, LSU):
    tilesPerCU = float(nBatches * Tiles0 * Tiles1) / (float(nCUs) / float(GSU) / float(LSU))
    return tilesPerCU

def CuGranularity (TilesPerCu):
    cuGranularity = float(TilesPerCu) / float(math.ceil(TilesPerCu))
    return cuGranularity

def ComputeGranularity(MT0, MT1, M, N, nCUs, GSU, LSU, nBatches):
    nTiles0 = nTiles(MT0, M)
    nTiles1 = nTiles(MT1, N)
    tileGranularity0 = TileGranularity(MT0,nTiles0,M)
    tileGranularity1 = TileGranularity(MT1,nTiles1,N)
    tilesPerCU = TilesPerCu(nBatches,nTiles0,nTiles1,nCUs,GSU,LSU)
    cuGranularity = CuGranularity(tilesPerCU)
    #return (nTiles0,nTiles1,tileGranularity0,tileGranularity1,tilesPerCU,cuGranularity)
    return cuGranularity

In [6]:
def WavesPerSolution(WG0,WG1):
    wavesPerSolution = (WG0*WG1)/64
    return wavesPerSolution

def WavePerSimd(tilesPerCU, wavesPerSolution):
    wavesPerSimd = (tilesPerCU * wavesPerSolution) / 4
    return wavesPerSimd

def WaveGranularity(wavesPerSimd):
    waveGranularity = float(wavesPerSimd) / float(math.ceil(wavesPerSimd))
    return waveGranularity

def ComputeWaveGranularity(WG0, WG1, MT0, MT1, M, N, nCUs, GSU, LSU, nBatches):
    nTiles0 = nTiles(MT0, M)
    nTiles1 = nTiles(MT1, N)
    tileGranularity0 = TileGranularity(MT0,nTiles0,M)
    tileGranularity1 = TileGranularity(MT1,nTiles1,N)
    tilesPerCU = TilesPerCu(nBatches,nTiles0,nTiles1,nCUs,GSU,LSU)
    wavesPerSolution = WavesPerSolution(WG0,WG1)
    wavesPerSimd = WavePerSimd(tilesPerCU, wavesPerSolution)
    waveGranularity = WaveGranularity(wavesPerSimd)
    return waveGranularity

In [7]:
icols_NN = dataNN.columns[:10]
solutionNames_NN=dataNN.columns[10:]
solutionsList_NN = solutionsD_NN[2:]
df_NN = pd.DataFrame(solutionsList_NN, index=solutionNames_NN)
df1_NN = df_NN[['MacroTile0','MacroTile1','ThreadTile0','ThreadTile1','WorkGroup','GlobalSplitU']]
df2_NN = pd.DataFrame(df1_NN['WorkGroup'].values.tolist(), columns=['WorkGroup0','WorkGroup1','LSU'], index=solutionNames_NN)
df3_NN = pd.concat([df1_NN.drop(['WorkGroup'],axis=1),df2_NN], axis=1)

icols_NT = dataNT.columns[:10]
solutionNames_NT=dataNT.columns[10:]
solutionsList_NT = solutionsD_NT[2:]
df_NT = pd.DataFrame(solutionsList_NT, index=solutionNames_NT)
df1_NT = df_NT[['MacroTile0','MacroTile1','ThreadTile0','ThreadTile1','WorkGroup','GlobalSplitU']]
df2_NT = pd.DataFrame(df1_NT['WorkGroup'].values.tolist(), columns=['WorkGroup0','WorkGroup1','LSU'], index=solutionNames_NT)
df3_NT = pd.concat([df1_NT.drop(['WorkGroup'],axis=1),df2_NT], axis=1)

icols_TN = dataTN.columns[:10]
solutionNames_TN=dataTN.columns[10:]
solutionsList_TN = solutionsD_TN[2:]
df_TN = pd.DataFrame(solutionsList_TN, index=solutionNames_TN)
df1_TN = df_TN[['MacroTile0','MacroTile1','ThreadTile0','ThreadTile1','WorkGroup','GlobalSplitU']]
df2_TN = pd.DataFrame(df1_TN['WorkGroup'].values.tolist(), columns=['WorkGroup0','WorkGroup1','LSU'], index=solutionNames_TN)
df3_TN = pd.concat([df1_TN.drop(['WorkGroup'],axis=1),df2_TN], axis=1)

In [8]:
df3_NT.head()

,MacroTile0,MacroTile1,ThreadTile0,ThreadTile1,GlobalSplitU,WorkGroup0,WorkGroup1,LSU
Cijk_Ailk_Bjlk_SB_MT256x32x16_SE_EPS1_GRVW1_K1_NLCA1_PGR1_WG32_4_2_WGM8,256,32,8,8,1,32,4,2
Cijk_Ailk_Bjlk_SB_MT256x32x32_SE_EPS0_GRVW1_K1_NLCA2_PGR0_WG32_4_1_WGM1,256,32,8,8,1,32,4,1
Cijk_Ailk_Bjlk_SB_MT256x32x32_SE_EPS0_GRVW1_K1_NLCA2_PGR0_WG32_4_1_WGM8,256,32,8,8,1,32,4,1
Cijk_Ailk_Bjlk_SB_MT256x32x16_SE_EPS0_GRVW4_K1_NLCA1_PGR0_WG32_4_1_WGM1,256,32,8,8,1,32,4,1
Cijk_Ailk_Bjlk_SB_MT256x32x32_SE_EPS0_GRVW4_K1_NLCA1_PGR0_WG32_4_1_WGM1,256,32,8,8,1,32,4,1


In [9]:
sizes_NN=dataNN[icols_NN][['SizeI','SizeJ','SizeK','SizeL']]
sizes_NT=dataNT[icols_NT][['SizeI','SizeJ','SizeK','SizeL']]
sizes_TN=dataTN[icols_TN][['SizeI','SizeJ','SizeK','SizeL']]

In [10]:
sizes_NN.head()

,SizeI,SizeJ,SizeK,SizeL
0,192,240,1,256
1,192,256,1,256
2,192,300,1,256
3,193,240,1,256
4,193,256,1,256


In [11]:
dfTemp = pd.DataFrame()
for key in range(0,len(sizes_NN)):
#for key in range(0,100):
    size = sizes_NN.iloc[key]
    cname = (size[0],size[1],size[2],size[3])
    
    dfTemp[cname] = df3_NN.apply(lambda row: ComputeGranularity(row.MacroTile0, row.MacroTile1, size[0], size[1], 60, row.GlobalSplitU, row.LSU, size[2]), axis=1)

In [12]:
dfTemp

,"(192, 240, 1, 256)","(192, 256, 1, 256)","(192, 300, 1, 256)","(193, 240, 1, 256)","(193, 256, 1, 256)","(193, 300, 1, 256)","(194, 240, 1, 256)","(194, 256, 1, 256)","(194, 300, 1, 256)","(195, 240, 1, 256)",...,"(316, 300, 1, 256)","(317, 240, 1, 256)","(317, 256, 1, 256)","(317, 300, 1, 256)","(318, 240, 1, 256)","(318, 256, 1, 256)","(318, 300, 1, 256)","(319, 240, 1, 256)","(319, 256, 1, 256)","(319, 300, 1, 256)"
Cijk_Ailk_Bljk_SB_MT256x32x8_SE_EPS0_GRVW1_K1_LPB4_NLCA2_PGR0_WG32_4_1_WGM1,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,...,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW1_K1_LPB0_NLCA2_PGR0_WG32_4_1_WGM1,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,...,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333
Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW4_K1_LPB0_NLCA1_PGR0_WG32_4_2_WGM1,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,...,0.666667,0.533333,0.533333,0.666667,0.533333,0.533333,0.666667,0.533333,0.533333,0.666667
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW4_K1_LPB4_NLCA1_PGR0_WG32_4_2_WGM8,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,...,0.666667,0.533333,0.533333,0.666667,0.533333,0.533333,0.666667,0.533333,0.533333,0.666667
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW1_K1_LPB4_NLCA1_PGR0_WG32_4_2_WGM8,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,...,0.666667,0.533333,0.533333,0.666667,0.533333,0.533333,0.666667,0.533333,0.533333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW4_K1_LPB4_NLCA1_PGR0_WG32_4_1_WGM8,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,...,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333
Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW1_K1_LPB0_NLCA2_PGR0_WG32_4_1_WGM8,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,...,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333
Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW4_K1_LPB0_NLCA1_PGR0_WG32_4_1_WGM8,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,...,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW1_K1_LPB0_NLCA1_PGR0_WG32_4_2_WGM1,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,...,0.666667,0.533333,0.533333,0.666667,0.533333,0.533333,0.666667,0.533333,0.533333,0.666667


In [13]:
dfTempWG = pd.DataFrame()
for key in range(0,len(sizes_NN)):
    size = sizes_NN.iloc[key]
    cname = (size[0],size[1],size[2],size[3])
    
    dfTempWG[cname] = df3_NN.apply(lambda row: ComputeWaveGranularity(row.WorkGroup0, row.WorkGroup1, row.MacroTile0, row.MacroTile1, size[0], size[1], 60, row.GlobalSplitU, row.LSU, size[2]), axis=1)

In [14]:
dfTempWG

,"(192, 240, 1, 256)","(192, 256, 1, 256)","(192, 300, 1, 256)","(193, 240, 1, 256)","(193, 256, 1, 256)","(193, 300, 1, 256)","(194, 240, 1, 256)","(194, 256, 1, 256)","(194, 300, 1, 256)","(195, 240, 1, 256)",...,"(316, 300, 1, 256)","(317, 240, 1, 256)","(317, 256, 1, 256)","(317, 300, 1, 256)","(318, 240, 1, 256)","(318, 256, 1, 256)","(318, 300, 1, 256)","(319, 240, 1, 256)","(319, 256, 1, 256)","(319, 300, 1, 256)"
Cijk_Ailk_Bljk_SB_MT256x32x8_SE_EPS0_GRVW1_K1_LPB4_NLCA2_PGR0_WG32_4_1_WGM1,0.066667,0.066667,0.083333,0.066667,0.066667,0.083333,0.066667,0.066667,0.083333,0.066667,...,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW1_K1_LPB0_NLCA2_PGR0_WG32_4_1_WGM1,0.066667,0.066667,0.083333,0.066667,0.066667,0.083333,0.066667,0.066667,0.083333,0.066667,...,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667
Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW4_K1_LPB0_NLCA1_PGR0_WG32_4_2_WGM1,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,...,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW4_K1_LPB4_NLCA1_PGR0_WG32_4_2_WGM8,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,...,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW1_K1_LPB4_NLCA1_PGR0_WG32_4_2_WGM8,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,...,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW4_K1_LPB4_NLCA1_PGR0_WG32_4_1_WGM8,0.066667,0.066667,0.083333,0.066667,0.066667,0.083333,0.066667,0.066667,0.083333,0.066667,...,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667
Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW1_K1_LPB0_NLCA2_PGR0_WG32_4_1_WGM8,0.066667,0.066667,0.083333,0.066667,0.066667,0.083333,0.066667,0.066667,0.083333,0.066667,...,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667
Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW4_K1_LPB0_NLCA1_PGR0_WG32_4_1_WGM8,0.066667,0.066667,0.083333,0.066667,0.066667,0.083333,0.066667,0.066667,0.083333,0.066667,...,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW1_K1_LPB0_NLCA1_PGR0_WG32_4_2_WGM1,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,...,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333


In [15]:
dfGran_NN = pd.concat([df3_NN, dfTemp],axis=1)
dfGranWG_NN = pd.concat([df3_NN, dfTempWG],axis=1)

In [16]:
dfGranWG_NN.head()

,MacroTile0,MacroTile1,ThreadTile0,ThreadTile1,GlobalSplitU,WorkGroup0,WorkGroup1,LSU,"(192, 240, 1, 256)","(192, 256, 1, 256)",...,"(316, 300, 1, 256)","(317, 240, 1, 256)","(317, 256, 1, 256)","(317, 300, 1, 256)","(318, 240, 1, 256)","(318, 256, 1, 256)","(318, 300, 1, 256)","(319, 240, 1, 256)","(319, 256, 1, 256)","(319, 300, 1, 256)"
Cijk_Ailk_Bljk_SB_MT256x32x8_SE_EPS0_GRVW1_K1_LPB4_NLCA2_PGR0_WG32_4_1_WGM1,256,32,8,8,1,32,4,1,0.066667,0.066667,...,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW1_K1_LPB0_NLCA2_PGR0_WG32_4_1_WGM1,256,32,8,8,1,32,4,1,0.066667,0.066667,...,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667,0.133333,0.133333,0.166667
Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW4_K1_LPB0_NLCA1_PGR0_WG32_4_2_WGM1,256,32,8,8,1,32,4,2,0.133333,0.133333,...,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW4_K1_LPB4_NLCA1_PGR0_WG32_4_2_WGM8,256,32,8,8,1,32,4,2,0.133333,0.133333,...,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW1_K1_LPB4_NLCA1_PGR0_WG32_4_2_WGM8,256,32,8,8,1,32,4,2,0.133333,0.133333,...,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333


In [17]:
dfGran_NN.head()

,MacroTile0,MacroTile1,ThreadTile0,ThreadTile1,GlobalSplitU,WorkGroup0,WorkGroup1,LSU,"(192, 240, 1, 256)","(192, 256, 1, 256)",...,"(316, 300, 1, 256)","(317, 240, 1, 256)","(317, 256, 1, 256)","(317, 300, 1, 256)","(318, 240, 1, 256)","(318, 256, 1, 256)","(318, 300, 1, 256)","(319, 240, 1, 256)","(319, 256, 1, 256)","(319, 300, 1, 256)"
Cijk_Ailk_Bljk_SB_MT256x32x8_SE_EPS0_GRVW1_K1_LPB4_NLCA2_PGR0_WG32_4_1_WGM1,256,32,8,8,1,32,4,1,0.133333,0.133333,...,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW1_K1_LPB0_NLCA2_PGR0_WG32_4_1_WGM1,256,32,8,8,1,32,4,1,0.133333,0.133333,...,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333,0.266667,0.266667,0.333333
Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW4_K1_LPB0_NLCA1_PGR0_WG32_4_2_WGM1,256,32,8,8,1,32,4,2,0.266667,0.266667,...,0.666667,0.533333,0.533333,0.666667,0.533333,0.533333,0.666667,0.533333,0.533333,0.666667
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW4_K1_LPB4_NLCA1_PGR0_WG32_4_2_WGM8,256,32,8,8,1,32,4,2,0.266667,0.266667,...,0.666667,0.533333,0.533333,0.666667,0.533333,0.533333,0.666667,0.533333,0.533333,0.666667
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW1_K1_LPB4_NLCA1_PGR0_WG32_4_2_WGM8,256,32,8,8,1,32,4,2,0.266667,0.266667,...,0.666667,0.533333,0.533333,0.666667,0.533333,0.533333,0.666667,0.533333,0.533333,0.666667


In [18]:
#keysNN = []
keysNN = sizes_NN.apply(lambda row: (row.SizeI,row.SizeJ,row.SizeK,row.SizeL), axis=1)
data1_NN = pd.DataFrame.copy(dataNN)
data1_NN.index = list(keysNN)
dataT_NN =data1_NN.iloc[:,10:].T

In [19]:
myidx = dataT_NN.max().idxmax()

In [20]:
myidx

(319, 300, 1, 256)

In [21]:
dfGran_NN[myidx]

Cijk_Ailk_Bljk_SB_MT256x32x8_SE_EPS0_GRVW1_K1_LPB4_NLCA2_PGR0_WG32_4_1_WGM1     0.333333
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW1_K1_LPB0_NLCA2_PGR0_WG32_4_1_WGM1    0.333333
Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW4_K1_LPB0_NLCA1_PGR0_WG32_4_2_WGM1    0.666667
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW4_K1_LPB4_NLCA1_PGR0_WG32_4_2_WGM8    0.666667
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW1_K1_LPB4_NLCA1_PGR0_WG32_4_2_WGM8    0.666667
                                                                                  ...   
Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW4_K1_LPB4_NLCA1_PGR0_WG32_4_1_WGM8    0.333333
Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW1_K1_LPB0_NLCA2_PGR0_WG32_4_1_WGM8    0.333333
Cijk_Ailk_Bljk_SB_MT256x32x32_SE_EPS0_GRVW4_K1_LPB0_NLCA1_PGR0_WG32_4_1_WGM8    0.333333
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW1_K1_LPB0_NLCA1_PGR0_WG32_4_2_WGM1    0.666667
Cijk_Ailk_Bljk_SB_MT256x32x16_SE_EPS0_GRVW4_K1_LPB0_NLCA1_PGR0_WG32_4_2_WGM8    0.666667
Name: (319, 300, 1, 2

In [22]:
kerMaxIdx = dataT_NN.idxmax()

In [23]:
myKernel = kerMaxIdx[myidx]

In [24]:
myKernel

'Cijk_Ailk_Bljk_SB_MT256x32x8_SE_EPS1_GRVW4_K1_LPB4_NLCA1_PGR1_WG32_4_2_WGM1'

In [25]:
dfGran_NN.loc[myKernel][keysNN].max()

0.6666666666666666

In [26]:
#dfGran_NN.corr(dataT_NN)
cor_NN = [dfGran_NN[key].corr(dataT_NN[key]) for key in keysNN]
corWG_NN = [dfGranWG_NN[key].corr(dataT_NN[key]) for key in keysNN]

In [27]:
cor_NN[0:10]

[0.8930187803048987,
 0.9173525710772698,
 0.7596942887398832,
 0.8396997826337387,
 0.971538630253564,
 0.9137544537173566,
 0.776148916060435,
 0.9698710272919812,
 0.9659572807138413,
 0.9446278893495743]

In [28]:
corWG_NN[0:10]

[0.8930187803048987,
 0.9173525710772698,
 0.7596942887398832,
 0.8396997826337387,
 0.971538630253564,
 0.9137544537173566,
 0.776148916060435,
 0.9698710272919812,
 0.9659572807138413,
 0.9446278893495743]

In [29]:
labels_NN = pd.Series(cor_NN).nsmallest(10).index
labelsWG_NN = pd.Series(corWG_NN).nsmallest(10).index

In [30]:
pd.Series(cor_NN).nsmallest(10)

158    0.607034
61     0.654425
172    0.658038
377    0.676714
175    0.693771
297    0.711651
183    0.721004
198    0.723681
156    0.735133
240    0.735865
dtype: float64

In [31]:
pd.Series(corWG_NN).nsmallest(10)

158    0.607034
61     0.654425
172    0.658038
377    0.676714
175    0.693771
297    0.711651
183    0.721004
198    0.723681
156    0.735133
240    0.735865
dtype: float64

In [32]:
cor_NN[135]

0.9662994909043345

In [33]:
sidx = 158
dfGran_NN.loc[myKernel][keysNN[sidx]] #.iloc[:,sidx]

0.3333333333333333

In [34]:
keysNN[sidx]

(244, 300, 1, 256)

In [35]:
type(list(labels_NN)[0])

int

In [36]:
[(cor_NN[n],dfGran_NN.loc[myKernel][keysNN[n]]) for n in labels_NN]# [list(labels_NN)]

[(0.6070344029323065, 0.3333333333333333),
 (0.6544253659802913, 0.26666666666666666),
 (0.6580380433353965, 0.26666666666666666),
 (0.676713612981711, 0.6666666666666666),
 (0.693770660265056, 0.26666666666666666),
 (0.7116511483755238, 0.5333333333333333),
 (0.7210042015809501, 0.26666666666666666),
 (0.7236812968043929, 0.5333333333333333),
 (0.7351332437236722, 0.26666666666666666),
 (0.7358645647312614, 0.5333333333333333)]